# ME 139 Project Calculations

This notebook acts like a sandbox to try out various snippets of code/calculations used for the project.


In [1]:
from typing import Callable
from dataclasses import dataclass

import numpy as np
import numpy.typing as npt

Here, we'll define a few functions to help us with creating the appropriate matrices.


In [2]:
def T(alpha_n: float, b_n: float, theta_n: float, d_n: float) -> npt.NDArray[np.float64]:
    """
    Creates a transformation matrix for given values of alpha, b, theta, and d.
    """
    return np.array([
        [np.cos(alpha_n), -np.sin(alpha_n), 0, b_n],
        [np.cos(alpha_n) * np.sin(theta_n), np.cos(alpha_n) *
         np.cos(theta_n), -np.sin(alpha_n), -d_n * np.sin(alpha_n)],
        [np.sin(alpha_n) * np.sin(theta_n), np.sin(alpha_n) *
         np.cos(theta_n), np.cos(alpha_n), d_n * np.cos(alpha_n)],
        [0, 0, 0, 1]
    ])


Below, we'll define some global variables as well as the transformation matrices as functions of the joint angles.


In [3]:
# Constants
l_f = 0.45 # femur length
l_t = 0.45 # tibia length

# Transformation matrices
T_01 = lambda measured_theta_1: T(np.pi / 2, 0, measured_theta_1, 0)
T_12 = lambda measured_theta_2: T(0, l_t, measured_theta_2, 0)
T_23 = lambda measured_theta_3: T(0, l_f, -measured_theta_3, 0)

T_02 = lambda measured_theta_1, measured_theta_2: T_01(measured_theta_1) @ T_12(measured_theta_2)
T_03 = lambda measured_theta_1, measured_theta_2, measured_theta_3: T_01(measured_theta_1) @ T_12(measured_theta_2) @ T_23(measured_theta_3)

T_13 = lambda measured_theta_2, measured_theta_3: T_12(measured_theta_2) @ T_23(measured_theta_3)

# Rotation matrices (extracted from transformation matrices)
R_01 = lambda measured_theta_1: T_01(measured_theta_1)[:3, :3]
R_02 = lambda measured_theta_1, measured_theta_2: T_02(measured_theta_1, measured_theta_2)[:3, :3]

# Displacement vectors (extracted from transformation matrices)
D_13 = lambda measured_theta_2, measured_theta_3: T_13(measured_theta_2, measured_theta_3)[:3, 3]
D_23 = lambda measured_theta_3: T_23(measured_theta_3)[:3, 3]

Now, we can define some Jacobians for the end-effector.

In [4]:
J_v1 = lambda measured_theta_1, measured_theta_2, measured_theta_3: np.zeros(1)

J_v2 = lambda measured_theta_1, measured_theta_2, measured_theta_3: np.array([
    [np.cross(R_01(measured_theta_1)[:,2], R_01(measured_theta_1) @ D_13(measured_theta_2, measured_theta_3))]
])

J_v3 = lambda measured_theta_1, measured_theta_2, measured_theta_3: np.array([
    [
        np.cross(R_01(measured_theta_1)[:,2], R_01(measured_theta_1) @ D_13(measured_theta_2, measured_theta_3)),
        np.cross(R_02(measured_theta_1, measured_theta_2)[:,2], R_02(measured_theta_1, measured_theta_2) @ D_23(measured_theta_3))
    ]
])

J_omega1 = lambda measured_theta_1, measured_theta_2, measured_theta_3: np.zeros(1)

J_omega2 = lambda measured_theta_1, measured_theta_2, measured_theta_3: np.array([
    [R_01(measured_theta_1)[:,2]]
])

J_omega3 = lambda measured_theta_1, measured_theta_2, measured_theta_3: np.array([
    [R_01(measured_theta_1)[:,2], R_02(measured_theta_1, measured_theta_2)[:,2]]
])

We can now define the inertia matrix for the entire system.

In [6]:
@dataclass
class MParams:
    measured_theta_1: float
    measured_theta_2: float
    measured_theta_3: float
    m_1: float
    m_2: float
    m_3: float
    I_c1: npt.NDArray[np.float64]
    I_c2: npt.NDArray[np.float64]
    I_c3: npt.NDArray[np.float64]

def M(*params: MParams):
    measured_theta_1, measured_theta_2, measured_theta_3, m_1, m_2, m_3, I_c1, I_c2, I_c3 = params
    
    term_2 = m_2 * J_v2(measured_theta_1, measured_theta_2, measured_theta_3).T @ J_v2(measured_theta_1, measured_theta_2, measured_theta_3) + \
        J_omega2(measured_theta_1, measured_theta_2, measured_theta_3).T @ I_c2 @ J_omega2(measured_theta_1, measured_theta_2, measured_theta_3)
    
    term_3 = m_3 * J_v3(measured_theta_1, measured_theta_2, measured_theta_3).T @ J_v3(measured_theta_1, measured_theta_2, measured_theta_3) + \
        J_omega3(measured_theta_1, measured_theta_2, measured_theta_3).T @ I_c3 @ J_omega3(measured_theta_1, measured_theta_2, measured_theta_3)
    
    return term_2 + term_3
